In the first part of this notebook (until the next Markdown), I mirror Feng's setup in baseline.ipynb for aggregating student data into single rows for each student (per module per presentation).

In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns

In [2]:
# Here I am using the local file name in the shared DSCB Dropbox folder

df = pd.read_csv('/Users/jamesemcnally/Dropbox/DSBC Student Risk Factors Datasets/Streaming_OULAD_dataset.csv')

In [3]:
df.sample(5)

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,...,page,questionnaire,quiz,repeatactivity,resource,sharedsubpage,subpage,url,score,final_result
622365,3.0,0.0,535799,1.0,7.0,1.0,1.0,1.0,0,60,...,0,0,0,0,14,0,15,1,NaN,2.0
1716548,6.0,3.0,694470,1.0,3.0,1.0,4.0,0.0,0,30,...,0,0,0,0,0,0,0,0,80.0,2.0
669549,3.0,1.0,567304,0.0,10.0,1.0,4.0,0.0,0,60,...,0,0,0,0,1,0,4,0,NaN,2.0
767954,3.0,1.0,2383036,1.0,9.0,2.0,1.0,0.0,0,120,...,0,0,0,0,1,0,3,0,NaN,2.0
1345948,5.0,1.0,597787,0.0,10.0,2.0,2.0,1.0,0,60,...,0,0,0,0,0,0,8,0,NaN,3.0


In [4]:
dfs = df.drop(['dataplus','date','score'], axis=1)
student_feats = ['gender','region','highest_education','imd_band','age_band',
                 'num_of_prev_attempts','studied_credits','final_result']
tag = ['id_student','code_module','code_presentation']
X1 = dfs.drop(student_feats,axis=1).groupby(tag).sum()
X2y = dfs[tag+student_feats].groupby(tag).mean()
dfc = pd.concat([X1,X2y],axis=1)

In [5]:
dfc

disability  dualpane  externalquiz  \
id_student code_module code_presentation                                       
6516       0.0         3.0                       0.0         0             0   
8462       3.0         1.0                       0.0         0            12   
                       3.0                       0.0         0             0   
11391      0.0         1.0                       0.0         0             0   
23629      1.0         0.0                       0.0         0             0   
...                                              ...       ...           ...   
2698257    0.0         1.0                       0.0         0             0   
2698535    2.0         2.0                       0.0         0             0   
           4.0         1.0                       0.0         2             0   
2698577    1.0         3.0                       0.0         0             0   
2698588    1.0         3.0                       0.0         0             0   

                                          folder  forumng  glossary  homepage  \
id_student code_module code_presentation                                        
6516       0.0         3.0                     0      451         0       497   
8462       3.0         1.0                     0       36         0       196   
                       3.0                     0        2         0         7   
11391      0.0         1.0                     0      193         0       138   
23629      1.0         0.0                     0       87         0        36   
...                                          ...      ...       ...       ...   
2698257    0.0         1.0                     0      201         0       165   
2698535    2.0         2.0                     0      309         0        98   
           4.0         1.0                     0     1953         0       391   
2698577    1.0         3.0                     0      103         0       204   
2698588    1.0         3.0                     0       67         0       171   

                                          htmlactivity  oucollaborate  \
id_student code_module code_presentation                                
6516       0.0         3.0                           0              0   
8462       3.0         1.0                           0             27   
                       3.0                           0              1   
11391      0.0         1.0                           0              0   
23629      1.0         0.0                           0              0   
...                                                ...            ...   
2698257    0.0         1.0                           0              0   
2698535    2.0         2.0                           0              0   
           4.0         1.0                           0              0   
2698577    1.0         3.0                           0              5   
2698588    1.0         3.0                           0              7   

                                          oucontent  ...  subpage  url  \
id_student code_module code_presentation             ...                 
6516       0.0         3.0                     1505  ...      143  143   
8462       3.0         1.0                       64  ...      230   23   
                       3.0                        0  ...        0    0   
11391      0.0         1.0                      553  ...       32    5   
23629      1.0         0.0                        0  ...        5    0   
...                                             ...  ...      ...  ...   
2698257    0.0         1.0                      331  ...       44    5   
2698535    2.0         2.0                        6  ...       52    6   
           4.0         1.0                      773  ...       13   90   
2698577    1.0         3.0                      253  ...       38    4   
2698588    1.0         3.0                      233  ...       76    2   

                                  

Next I am measuring correlation ratios for each of the features against the "final grade" column as the target as part of EDA.

In [7]:
def correlation_ratio(categories, values):
    """
    Calculate eta-squared: proportion of variance explained by categories
    """
    categories = np.array(categories)
    values = np.array(values)
    
    # Overall mean
    mean_total = np.mean(values)
    
    # Between-group variance
    ss_between = sum(len(values[categories == cat]) * 
                     (np.mean(values[categories == cat]) - mean_total)**2 
                     for cat in np.unique(categories))
    
    # Total variance
    ss_total = sum((values - mean_total)**2)
    
    return ss_between / ss_total if ss_total != 0 else 0

# Calculate for all numeric features
results = []
for col in df.select_dtypes(include=[np.number]).columns:
    if col != 'final_result':
        eta = correlation_ratio(df['final_result'], df[col])
        results.append({'feature': col, 'eta_squared': eta})

eta_df = pd.DataFrame(results).sort_values('eta_squared', ascending=False)
print(eta_df)

                 feature  eta_squared
5      highest_education     0.026956
11                  date     0.017741
6               imd_band     0.013386
8   num_of_prev_attempts     0.012313
9        studied_credits     0.007915
7               age_band     0.006110
1      code_presentation     0.005489
16               forumng     0.005096
10            disability     0.004769
21             oucontent     0.001794
2             id_student     0.001697
0            code_module     0.001144
18              homepage     0.000758
23                ouwiki     0.000613
30               subpage     0.000587
19          htmlactivity     0.000531
24                  page     0.000487
25         questionnaire     0.000476
12              dataplus     0.000435
15                folder     0.000364
14          externalquiz     0.000228
26                  quiz     0.000214
3                 gender     0.000158
4                 region     0.000136
20         oucollaborate     0.000108
28          

Now I am adding these proportions to a summary table of zeros, non-zeros, nulls, and % of zeros for each column. 

The first summary table lists the "zero count," "non-zero count," "total non-null," and "% zero" for each column.

The second summary table compares the % non-zero to the ETA squared of each column, sorted descending by ETA squared.

In [9]:
summary = pd.DataFrame({
    'Zero Count': (df == 0).sum(),
    'Non-Zero Count': (df != 0).sum(),
    'Total Non-Null': df.count()
})
summary['% Zero'] = (summary['Zero Count'] / summary['Total Non-Null'] * 100).round(2)
summary['% Non-Zero'] = (summary['Non-Zero Count'] / summary['Total Non-Null'] * 100).round(2)

# Add eta-squared as percentage
summary['Eta Squared %'] = (eta_df.set_index('feature')['eta_squared'] * 100).round(2)

# Sort by importance
summary = summary.sort_values('Eta Squared %', ascending=False)

print(summary)

                      Zero Count  Non-Zero Count  Total Non-Null  % Zero  \
highest_education          11810         1707173         1718983    0.69   
date                        1339         1717644         1718983    0.08   
imd_band                  146487         1572496         1718983    8.52   
num_of_prev_attempts     1548009          170974         1718983   90.05   
studied_credits                0         1718983         1718983    0.00   
age_band                 1122808          596175         1718983   65.32   
code_presentation         280841         1438142         1718983   16.34   
forumng                   939201          779782         1718983   54.64   
disability               1564411          154572         1718983   91.01   
oucontent                 928399          790584         1718983   54.01   
id_student                     0         1718983         1718983    0.00   
code_module                62531         1656452         1718983    3.64   
homepage    

None of the individual interaction features seem to explain much of the variance in the final result, with "forumng" having the highest score at 0.51%. However:

1. I could be / am probably doing at least some of the math wrong. (Please suggest other ways to come at this problem!)
2. We have not aggregated the interaction data, which may dilute the effects of interaction in this calculation.